# <font color=red>LangChain:  Collaborative Agents to reason, e.g. to solve puzzles</font>
- https://docs.langchain.com/docs

<h4>
We are attempting to create a simplified version of agent-based systems such as ChatDev, MetaGPT, Autogen, etc.</br>
Our version will instantiate our own Agent class as needed, and the main will drive their interactions.
</h4>
<span style="font-family:'Comic Sans MS', cursive, sans-serif;"><font color=orange>
## Demo 1 - Example that uses 2 agents (solver and analyzer) to solve puzzle about using water jugs of various sizes.
</font></span></br>
The trick about this puzzle is that the LLM may want to use both the 12-liter and 6-liter jugs, but only the 6 is required.</br>

In [ ]:
import sys, os, time

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

class Agent:
    def __init__(self,name="agent", role="", llm=None, sys_msg=""):
        self.name = name
        self.role = role
        self.llm = llm
        self.memory = ConversationBufferMemory(memory_key="chat_history")
        assert sys_msg, "Each agent must be initialized with a system message."
        post_sys_msg = """
            Previous conversation:
            {chat_history}
            New query: {question}
            Response:
        """
        post_sys_msg = "\n".join([ l.strip() for l in post_sys_msg.split("\n") ])
        template = sys_msg + post_sys_msg
        self.prompt = PromptTemplate.from_template(template)
        self.conversation = LLMChain(llm=self.llm, prompt=self.prompt,
                                     memory=self.memory, verbose=False)

    def run(self,query):
        response = self.conversation({"question": query})
        print("RESPONSE FROM",self.name)
        print(response["text"])
        print("-" * 50)
        return response["text"]

#### setup the solver agent
solver_sys_msg = """
You are an excellent problem solver.
You suggest solutions to problems that require as few steps as possible.
You also may have an iterative conversation with analyzer1 who probes you with questions
to help determine if there may be a solution with fewer steps.
If you are responding to an analysis of a solution and you fully agree with analyzers's
proposal but do not make any changes based on it, then say that and then print:  ALL_AGREE
"""

solver_llm = ChatOpenAI(model_name="gpt-4", temperature=0.0, max_tokens=256)

solver1 = Agent(
    name="solver1",
    role="Propose solutions to small puzzles",
    llm=solver_llm,
    sys_msg=solver_sys_msg,
)


#### setup the analyzer agent
analyzer_sys_msg = """
You are an excellent problem solving helper.
You do not suggest solutions yourself, but you question solver1 about any potential
solutions that it has developed, probing for ways to improve its solution by reducing
the number of steps.
If you have a better idea, you can pose it as a question.
If you do not want to suggest changes to solver1's proposed solution, say that and then
print:  ALL_AGREE
"""

analyzer_llm = ChatOpenAI(model_name="gpt-4", temperature=0.0, max_tokens=256)

analyzer1 = Agent(
    name="analyzer1",
    role="Analyze solutions from solver1",
    llm=analyzer_llm,
    sys_msg=analyzer_sys_msg,
)


#### now do the solve-and-analyze loop
user_query = """
If I have two empty jugs, a 12-liter jug and a 6-liter jug, what is a way to measure and
pour exactly 6 liters into a barrel using the fewest possible number of steps?
"""

analyzer_response = user_query  # prime for loops
NUM_ITERS = 3  # may put in sys.argv later
for iteridx in range(NUM_ITERS):
    solver_response = solver1.run(analyzer_response)
    if not solver_response  or  "ALL_AGREE" in solver_response:
        break
    analyzer_response = analyzer1.run(solver_response)
    if not analyzer_response  or  "ALL_AGREE" in analyzer_response:
        break